In [1]:
import networkx as nx

import pandas as pd
import numpy as np
import scipy
import random
import matplotlib.pyplot as plt
from itertools import combinations, groupby
#from IPython.core.display import HTML
import matplotlib.patches as mpatches
import gurobipy as gp
import time 

import helpers
import utils

start_time=time.time()


In [2]:
k = 2 # Number of data points (initial maps) we want in our simulation, for given data in this cell.
# These are generated randomly, larger the better.


datafile1 = '20_20_10.txt' #for standard maps with lower compactness scores


Egap = 0.20 #the bound we want to impose for fairenss
nodesx = 20 # for grid graphs: used in visualization as well
nodesy = 20 # for grid graphs
NumberOfUnits = nodesx*nodesy #total number of units
NumberOfDistricts = 10 #Number of districts we want

minbound, maxbound = 350, 400 #Population bounds for each unit, sampled uniformly
distmin, distmax = 0.1, 0.9 #Vote share fraction bounds for each unit, for party A, sampled uniformly
maxiter = 100 #Number of maps we want to have in our pool, used by the algorithm

budgetB = 400 #Party B's budget, spent uniformly 
#Can also be made to match with A's. This edit can be made in the next cell.

budgetrange = [100,300,500,700,900]#,  300, 500] #A's budget range
N, G = utils.gridgraph(nodesx,nodesy)

alphaset = [0.5]#, 0.7, 0.9] #Alpha: the baseline voter turnout factor. 

V, P0, Q0 = utils.RandomPopulation(NumberOfUnits, minbound, maxbound,distmin, distmax,  majority = True)

votesAoriginal = np.multiply(V,P0)
votesBoriginal = np.multiply(V,Q0)


#print(votesAoriginal, votesBoriginal)
VSA = sum(votesAoriginal)/(sum(votesAoriginal)+sum(votesBoriginal))
print('Voteshare A:', VSA)
print('Voteshare B:', 1-VSA)

print(utils.moranI(G, P0))

Voteshare A: 0.509352421692704
Voteshare B: 0.490647578307296
-0.033974132981440586


In [3]:
datafile = datafile1
df0 = pd.read_csv(datafile, index_col=False, header=None)


d = {i:[] for i in alphaset}

for alpha in alphaset:
    boxlistSeats1 = []
    boxlistInfeasibles1=[]
    votesA = alpha*votesAoriginal
    votesB = alpha*votesBoriginal

    TotalVotes = sum(votesA)+ sum(votesB)
    rows, cols = df0.shape
    
    Numrange = [] #10396
    
    while len(Numrange)<k:
      initmapnumber = random.randint(0,rows)
      v1 = df0.iloc[initmapnumber].to_numpy()
      W = 0 
      
      for i in range(0,NumberOfDistricts):
        ind = np.where(v1==i)
        if np.sum(votesA[ind]) > np.sum(votesB[ind]):

          W = W+ (3*np.sum(votesB[ind])-np.sum(votesA[ind]))/2
        else:
          W = W + (np.sum(votesB[ind])-3*np.sum(votesA[ind]))/2
        


      EgapCurrent=(W/TotalVotes)
      if abs(EgapCurrent) < Egap:
        Numrange.append(initmapnumber)
        
    print(EgapCurrent, 'initial map egap')
    i=0
    print(k, ' maps generated for alpha = ', alpha)

    
    for budgetA in budgetrange:

      #budgetB=budgetA
      difflist=[]
      InfList = []
      Targetrange =[]
      for num in Numrange:
        initmapnumber = num 
        EgapTotal, initmap, dfupdated, targetmap, Max, MaxNOstrat, MaxStrat, rd1wins, rd2wins, EgapRd2, InfNumber, YesNumber, targetmapnumber, NewvotesA, NewvotesB = helpers.Doeverything(datafile, initmapnumber, V, votesA, votesB, Egap, budgetA, budgetB, alpha, maxiter, NumberOfDistricts)
        difflist.append(MaxStrat-MaxNOstrat)
        #difflist.append(MaxStrat)
        InfList.append(InfNumber)
        Targetrange.append(targetmapnumber)
      i=i+1
      boxlistSeats1.append(difflist)
      boxlistInfeasibles1.append(InfList)
    d[alpha] = boxlistSeats1
    print(boxlistSeats1)

0.09347485800338803 initial map egap
2  maps generated for alpha =  0.5
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-11
[[2, 1], [3, 3], [3, 4], [4, 4], [5, 5]]


In [4]:
dfoo = pd.DataFrame(d[alphaset[0]])
dfoo.to_excel("outputA.xlsx",
         sheet_name='Alpha = %s' % alphaset[0]) 
for alpha in alphaset:
  if alpha != alphaset[0]:
    dfoo = pd.DataFrame(d[alpha])
    with pd.ExcelWriter("outputA.xlsx", engine="openpyxl", mode="a") as writer: 
        dfoo.to_excel(writer, sheet_name='Alpha = %s' % alpha)
VotesOriginal = [votesAoriginal, votesBoriginal]
maps = [Numrange, Targetrange]
dfoo =  pd.DataFrame(VotesOriginal)
with pd.ExcelWriter("outputA.xlsx", engine="openpyxl", mode="a") as writer: 
        dfoo.to_excel(writer, sheet_name='VoteShares')
dfoo =  pd.DataFrame(maps)
with pd.ExcelWriter("outputA.xlsx", engine="openpyxl", mode="a") as writer: 
        dfoo.to_excel(writer, sheet_name='MapsInfo_ %s' % alpha)


In [5]:
datafile = datafile1

df0 = pd.read_csv(datafile, index_col=False, header=None)
budgetA = 400 #Party B's budget, spent uniformly 
#Can also be made to match with A's. This edit can be made in the next cell.

budgetrange = [100,300,500,700,900]#,  300, 500] #A's budget range

d2 = {i:[] for i in alphaset}

for alpha in alphaset:

    boxlistSeats2 = []
    boxlistInfeasibles2=[]
    votesA = alpha*votesAoriginal
    votesB = alpha*votesBoriginal

    TotalVotes = sum(votesA)+ sum(votesB)
    rows, cols = df0.shape

    Numrange2 = []
    while len(Numrange2)<k:
      initmapnumber = random.randint(0,rows)
      v1 = df0.iloc[initmapnumber].to_numpy()
      W = 0 
      for i in range(0,NumberOfDistricts):
        ind = np.where(v1==i)
        if np.sum(votesA[ind])> np.sum(votesB[ind]):

          W = W+ (3*np.sum(votesB[ind])-np.sum(votesA[ind]))/2
        else:
          W = W + (np.sum(votesB[ind])-3*np.sum(votesA[ind]))/2

      EgapCurrent=(W/TotalVotes)
      if abs(EgapCurrent) < Egap:
        Numrange2.append(initmapnumber)

    i=0
    print(k, ' maps generated for alpha = ', alpha)

    
    for budgetB in budgetrange:
      #budgetB=budgetA
      difflist=[]
      InfList = []
      Targetrange2 = []
      for num in Numrange2: 
        initmapnumber = num
        EgapTotal, initmap, dfupdated, targetmap, Max, MaxNOstrat, MaxStrat, rd1wins, rd2wins, EgapRd2, InfNumber, YesNumber, targetmapnumber, NewvotesA, NewvotesB = helpers.Doeverything(datafile, initmapnumber, V, votesA, votesB, Egap, budgetA, budgetB, alpha, maxiter, NumberOfDistricts)
        difflist.append(MaxStrat-MaxNOstrat)
        #difflist.append(MaxStrat)
        InfList.append(InfNumber)
        Targetrange2.append(targetmapnumber)
      i=i+1
      boxlistSeats2.append(difflist)
      boxlistInfeasibles2.append(InfList)
    d2[alpha] = boxlistSeats2
    print(boxlistSeats2)

2  maps generated for alpha =  0.5
[[3, 3], [3, 4], [2, 3], [3, 3], [3, 4]]


In [6]:
dfoo = pd.DataFrame(d2[alphaset[0]])
dfoo.to_excel("outputB.xlsx",
             sheet_name='Sheet_name_%s' % alphaset[0]) 
for alpha in alphaset:
    if alpha != alphaset[0]:
        dfoo = pd.DataFrame(d2[alpha])
        with pd.ExcelWriter("outputB.xlsx", engine="openpyxl", mode="a") as writer: 
            dfoo.to_excel(writer, sheet_name='Sheet_name_%s' % alpha)
VotesOriginal = [votesAoriginal, votesBoriginal, V]
dfoo = pd.DataFrame(VotesOriginal)
with pd.ExcelWriter("outputB.xlsx", engine="openpyxl", mode="a") as writer: 
        dfoo.to_excel(writer, sheet_name='VoteShares')
maps = [Numrange2, Targetrange2]
dfoo =  pd.DataFrame(maps)
with pd.ExcelWriter("outputB.xlsx", engine="openpyxl", mode="a") as writer: 
        dfoo.to_excel(writer, sheet_name='MapsInfo_ %s' % alpha)


In [7]:
end_time = time.time()
print("Total_time = ", end_time-start_time, " seconds")

Total_time =  15.32720398902893  seconds
